# Geospatial Heatmap Data Visualization

Configure paths and column names:

In [1]:
import os
import json
import numpy as np
import pandas as pd
import folium
from folium.features import GeoJsonTooltip

# paths
OUT_DIR      = os.path.abspath(os.path.join(".", "out"))
IN_PATH      = os.path.join(OUT_DIR, "subphenotypes.csv")
GEOJSON_PATH = os.path.abspath(os.path.join(".", "data", "assets", "districts.geojson"))

# column names
SCORE_COL    = "GLP1_Focused_Priority_Score"
DISTRICT_CSV = "District"
STATE_CSV    = "State/UT"

# geojson field names
DISTRICT_GJ  = "district"
STATE_GJ     = "st_nm"

# output path
OUT_HTML     = os.path.join(OUT_DIR, "heatmap.html")

Construct alias mapping:

In [2]:
# alias map to reconcile common spelling variants between CSV and GeoJSON
ALIASES = {
    "aizawl":  "aizawal",
    "belgaum": "belagavi",
    "aravali": "aravalli"
}

Construct lookup:

In [3]:
def norm(s:str) -> str:
    if s is None: return ""
    
    return str(s).strip().lower()


df = pd.read_csv(IN_PATH)

# build normalized matching keys
df["district_key"] = df[DISTRICT_CSV].map(norm).map(lambda x: ALIASES.get(x, x))
df["state_key"]    = df[STATE_CSV].map(norm)

# load .geojson
with open(GEOJSON_PATH, "r", encoding="utf-8") as f:
    geo = json.load(f)

# build a lookup from (district_key, state_key) -> score
csv_lookup = { (r["district_key"], r["state_key"]): r[SCORE_COL] for _, r in df.dropna(subset=[SCORE_COL]).iterrows() }

print(csv_lookup)

{('adilabad', 'telangana'): 115.65736135380602, ('agar malwa', 'madhya pradesh'): -71.67599390937184, ('agra', 'uttar pradesh'): -50.83056890837205, ('ahmadabad', 'gujarat'): 16.46523168895667, ('ahmadnagar', 'maharashtra'): -2.09809025856855, ('aizawal', 'mizoram'): 31.567590832021537, ('ajmer', 'rajasthan'): -85.02986059065623, ('akola', 'maharashtra'): -1.7197868256853606, ('alappuzha', 'kerala'): 235.93595803948332, ('aligarh', 'uttar pradesh'): -50.1414952480193, ('alirajpur', 'madhya pradesh'): -70.00840886253499, ('allahabad', 'uttar pradesh'): -49.72247768442658, ('almora', 'uttarakhand'): 21.88136972557858, ('alwar', 'rajasthan'): -81.89716296517489, ('ambala', 'haryana'): 55.20743976867241, ('ambedkar nagar', 'uttar pradesh'): -49.9667404622019, ('amethi', 'uttar pradesh'): -50.29013476037388, ('amravati', 'maharashtra'): -2.186005603088592, ('amreli', 'gujarat'): 15.917848930799655, ('amritsar', 'punjab'): 92.64179906974545, ('anand', 'gujarat'): 16.609794871367907, ('ananta

Match priority scores:

In [4]:
for feat in geo["features"]:
    props = feat.get("properties", {})
    
    d = norm(props.get(DISTRICT_GJ, ""))
    s = norm(props.get(STATE_GJ, ""))

    # match score
    score = csv_lookup.get((d, s))
    
    props["glp1_score"] = float(score) if score else None
    feat["properties"] = props

In [5]:
from branca.colormap import linear
import numpy as np  # for np.isnan

values     = df[SCORE_COL].dropna()
VMIN, VMAX = float(values.min()), float(values.max())

# color map palette
cmap         = linear.YlOrRd_09.scale(VMIN, VMAX)
cmap.caption = "GLP1-Focused Priority Score"


def style_fn(feature):
    val = feature["properties"].get("glp1_score")
    
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return { "fillOpacity": 0.2, "weight": 0.2, "fillColor": "#cccccc", "color": "#555555" }
    
    return {
        "fillOpacity": 0.8,
        "weight":      0.3,
        "fillColor":   cmap(float(val)),
        "color":       "#333333",
    }

# build map
m = folium.Map(location=[22.9734, 78.6569], zoom_start=5, tiles="cartodbpositron")

folium.GeoJson(
    geo,
    name="GLP1 Priority by District",
    style_function=style_fn,
    tooltip=GeoJsonTooltip(
        fields=[DISTRICT_GJ, STATE_GJ, "glp1_score"],
        aliases=["District", "State/UT", "GLP1 Score"],
        sticky=True,
        labels=True,
    ),
).add_to(m)

# add the legend
cmap.add_to(m)

folium.LayerControl().add_to(m)

m.save(OUT_HTML)

print(f"Saved: {OUT_HTML}")

Saved: /home/aarush/Projects/Novo_Nordisk/subphenotype/out/heatmap.html
